In [20]:
import csv
import math
import heapq
import copy
from dataclasses import dataclass, field
from typing import Any
from itertools import count
import matplotlib.pyplot as plt
import numpy as np
from task import Task
from caseLoader import CaseLoader

In [21]:
# some constants used for indexing
NAME = 0
DURATION = 1
PERIOD = 2
TYPE = 3
PRIORITY = 4
DEADLINE = 5
RELEASE_TIME = 6
RESPONSE_TIME = 7 
WCRT = 8

MT = 10 # microtick

# https://docs.python.org/3/library/heapq.html
# use to only compare deadline for heapq invariant
# use identifer to return elements of same priority in order they were inserted
@dataclass(order=True)
class Task_not:
    deadline: int
    item: Any=field(compare=False) 
    identifier: int = field(default_factory=count().__next__)

In [26]:
ts1 = []
ts2 = []
TS = []
with open('test_cases/inf_10_10/taskset__1643188013-a_0.1-b_0.1-n_30-m_20-d_unif-p_2000-q_4000-g_1000-t_5__0__tsk.csv', newline='') as csvfile:
    testreader = csv.DictReader(csvfile, delimiter=';')
    for line in testreader:
        if line['type'] == "TT":
            #ts1.append([line['name'], int(line['duration']), int(line['period']), line['type'], int(line['priority']), int(line['deadline'])])
            ts2.append(Task_not(int(line['deadline']), [line['name'], int(line['duration']), int(line['period']), line['type'], int(line['priority']), int(line['deadline']), 0, None, 0]))
            TS.append(Task(line['name'], int(line['duration']), int(line['period']), 0, int(line['priority']), int(line['deadline'])))
#heapq.heappush(ts2, task(2000, ['tTT1', 4, 2000, 'TT', 7, 2000]))

TS[0].deadline

4000

In [27]:
#loader = CaseLoader()
#cases = loader.load_test_case("inf_30_30")


#cases

In [47]:
# sort ready list in increasing order on deadline, calling with 0 will give all
def get_ready1(task_set, cycle, ready_list):
    for task in task_set:
        if cycle % task.deadline == 0:
            new_job = copy.deepcopy(task)
            new_job.deadline = new_job.deadline * int(cycle/new_job.deadline) if cycle != 0 else new_job.deadline
            heapq.heappush(ready_list, new_job)
            
    return ready_list

def remove_from_ready(ready_list):
    heapq.heappop(ready_list)
    
    
def get_ready(task_set, cycle, ready_list):
    for task in task_set:
        if cycle % task.deadline == 0:
            new_job = copy.deepcopy(task)
            new_job.deadline = new_job.deadline * int(cycle/new_job.deadline) if cycle != 0 else new_job.deadline
            ready_list.append(task)
    
    # sort on deadline somehow
    # https://www.geeksforgeeks.org/sorting-objects-of-user-defined-class-in-python/
    return sorted(ready_list, key=lambda t: t.deadline)
        

# ts is set of TT tasks
def edf(ts):
    T = math.lcm(2000,3000,4000) # least common multiple of TT task periods. 2000 mt = 
    S = [] # schedule will be hyperperiod long. 12 000 microticks == 120 000 microsecs == 120 ms
    ready_list = []
    wcrts = {} # worst case response times
    print(T)

    for t in range(0, T):
        ready_list = get_ready(ts, t, ready_list)
        
        if ready_list == []:
            S[t] = "IDLE" 
            continue
        else:
            pass
        
        for task in ready_list:
            if task.duration > 0 and task.deadline <= t:
                return []
        
            # job done check response time gt wcrt and remove from ready list
            if task.duration == 0 and task.deadline >= t:
                response_time = t - task.release_time
                
                if response_time >= ts[t.name]:
                    wcrts[t.name] = response_time
                
                ready_list.remove(task)
                
            
        # EDF get next job to execute 
        S.append(ready_list[0].name)
        ready_list[0].duration = ready_list[0].duration - 1
        
        
    if ready_list != []:
        return []
    
    return S, wcrts

In [48]:
# sum wcrts as  cost??

s, ts = edf(TS)
#print(s)
#len(s)

#print(wcrts)

12000


AttributeError: 'Task' object has no attribute 'release_time'

In [ ]:
# sort ready list in increasing order on deadline, calling with 0 will give all
def get_ready(task_set, cycle, ready_list):
    for task in task_set:
        if cycle % task.deadline == 0:
            new_job = copy.deepcopy(task)
            new_job.deadline = new_job.deadline + cycle 
            heapq.heappush(ready_list, new_job)
            
    return ready_list

def remove_from_ready(ready_list):
    heapq.heappop(ready_list)

In [ ]:
def edf(ts):
    T = math.lcm(2000,3000,4000) # least common multiple of TT task periods. 2000 mt = 
    schedule = [] # schedule will be hyperperiod long. 12000 microticks == 120 000 microsecs == 120 ms
    ready_list = get_ready(ts, 0, [])
    t = 0
    
    while t < T:
        for t in ready_list:
            if t.item[DURATION] > 0 and t.
            
    


In [12]:
l = [1,2,3]

In [13]:
del l[1]

In [14]:
l

[1, 3]

NameError: name 'ts' is not defined